In [ ]:
import spacy
import pandas as pd
import re
import csv
import os
import time

In [ ]:
copyrightpath = os.getcwd() + "/YourCSVFile.csv"  ##Assuming that the File is hard-codedly provided
df = pd.read_csv(copyrightpath, error_bad_lines= False, delimiter="\t", quoting=csv.QUOTE_NONE)
data = df.iloc[:10000,:] ##Configured according to Big CSV file
data['copyright'] = data['Notice, this list of conditions, the following disclaimer, and the original OpenSSL and SSLeay Licences below.",f'].astype(str)
data['copyright'] = data['copyright'].str.lower()
nlp = spacy.load("en_core_web_sm")


In [ ]:
def entityCheck(listA, listB, index, hit_index):
    THE_PROBABLE_LOGIC_POS_CHECK = ['NOUN', 'NUM', 'PROPN', 'PROPN']
    THE_PROBABLE_LOGIC_NER_CHECK = ['DATE', 'PERSON', 'CARDINAL', 'ORG']
    if set(THE_PROBABLE_LOGIC_POS_CHECK).intersection(set(listA["POS_TAG"])):
        if set(THE_PROBABLE_LOGIC_NER_CHECK).intersection(set(listB["Values"])):
                for _values in listB["Values"]:
                    if THE_PROBABLE_LOGIC_NER_CHECK[0] in _values:
                        print("It is a copyright!!")
                        try:
                            hit_index.append(index)
                        except:
                            print(index)
                    elif THE_PROBABLE_LOGIC_NER_CHECK[2] in _values:
                        for _val in listB["Entity"]:
                            try:
                                pattern_regex = r'''^((19|20)\d{2}|\d{2})([,\s-]{1,3}((19|20)\d{2}|\d{1,2}))*$'''
                                extract_list = re.search(pattern_regex,_val)
                                if extract_list:
                                    print("It is a copyright!!")
                                    hit_index.append(index)
                            except:
                                print(index)

        else:
            print("It is a not a copyright!!")
    return hit_index

In [ ]:
def accuracyScore():
    counter = 0
    total_counter = 0
    for index, row in data.iterrows():
        original_tag = data.loc[index, 'new_tag']
        algorithm_tag = data.loc[index,'Hit&Miss']

        if original_tag == "t":
            total_counter += 1
            if algorithm_tag == original_tag:
                counter += 1

    accuracy_score = str((counter/total_counter) * 100) + " %"
    
    return accuracy_score

In [ ]:
def preProcessing():
    start = time.time()
    final_indexes = []
    temp_var = []
    hit_index = []

    ## Iterating through each row and doing preprocessing over it.
    ## Picking out the manual tags from the csv and putting them into seperate column "Original Tag"
    for index, row in data.iterrows():
        text = data.loc[index, 'copyright']
        text_split = text.split(",")
        data.loc[index,'new_tag'] = str(text_split[-1])

        doc = nlp(text)

        if type(text) == float:
            continue
        
        ## Lemmatization
        lemma_list = []
        for token in doc:
            lemma_list.append(token.lemma_)
        
        # Filter the stopword
        filtered_sentence =[] 
        for word in lemma_list:
            lexeme = nlp.vocab[word]
            if lexeme.is_stop == False:
                filtered_sentence.append(word)
        
        # Remove punctuation
        punctuations="?:!.,;"
        for word in filtered_sentence:
            if word in punctuations:
                filtered_sentence.remove(word)

        ## List joining and Filtering (c) and copyright unicode symbol
        list_of_copyrights = " ".join(map(str,filtered_sentence))
        substring = "( c )"
        cp_symbol = '\xa9' ##Unicode for copyright Symbol
        
        if "copyright" not in list_of_copyrights:
            if substring in list_of_copyrights:
                list_of_copyrights = list_of_copyrights.replace(substring, "copyright")
            
            elif cp_symbol in list_of_copyrights:
                list_of_copyrights = list_of_copyrights.replace(cp_symbol, "copyright")
    
        ## Implementing NER and POS Tags after normalization
        doc2 = nlp(list_of_copyrights)

        ## All the NER taggings will be contained in a dictionary having "Entity" and "Values" as keys
        ent_dict = {}

        full_table_ner = { "Entity": [], "Values": []}

        for x in doc2.ents:
            ent_dict[x.text] = x.label_

        for key in ent_dict:
            full_table_ner["Entity"].append(key)
            full_table_ner["Values"].append(ent_dict[key])
        
        ## All the POS taggings will be contained in a dictionary having "Entity" and "POS_TAGS" as keys
        pos_dict = {}
        full_table_pos = { "Entity": [], "POS_TAG": []}

        for token in doc:
            if not token.is_punct | token.is_space:
                pos_dict[token.text] = token.pos_

        for key in pos_dict:
            full_table_pos["Entity"].append(key)
            full_table_pos["POS_TAG"].append(pos_dict[key])

        ## The checking function call happening with each iteration
        entityCheck(full_table_pos, full_table_ner, index, hit_index)

    ## Updating the predicted TP results into one different column called "Hit&Miss"
    for i in hit_index:
        data.loc[i,'Hit&Miss'] = "t"

    data["Hit&Miss"].fillna("f", inplace=True)

    percentScore = accuracyScore()
    print(percentScore)
    expected_time = str((time.time() - start))
    print(expected_time + " sec")

    return

In [ ]:
preProcessing()

In [9]:
print(accuracyScore())

87.28056004308023 %
